### Some notes:

- Fixed size
- Contiguous block of memory
- O(1) to store and access an element
- Strings, stacks, queues, and hash tables use arrays for the implementation

### Limitation

- Looking up an element by value requires an entire traversal of the array (unless it's sorted)
- Deleting an element makes all subsequent elements have to be shifted left by one => O(n) time operation
- Similar to inserting
- Fixed size => should be careful of off-by-one errors
- Python does not have native support for arrays => use `list`, which dynamically resizes under the hood

## Problem 1: Get product of all other elements

Given an array of integers, return a new array such that each element at index i of the new array is the product of all the numbers in the original array except the one at i.   
For example: 
- If our input was \[ 1, 2, 3, 4, 5\], the expected output would be \[120, 60, 40, 30, 24\].
- If our input was \[3, 2, 1\], the expected output would be \[2, 3, 6\].

Follow-up: What if you can't use division?

#### Main Topic: Prefix and Suffix Product Arrays (Product of Array Except Self)

#### The Idea to Solve the Problem
We want each result[i] to be the product of all numbers in the array except nums[i] — but without using division.

#### Key idea:
- Use prefix products (product of all elements to the left).
- Use suffix products (product of all elements to the right).   
  => Use 2 loops, not 1
- Multiply them to get the final result.
- This gives O(n) time and O(1) extra space (if we reuse the result array).

#### Pseudo Code

In [ ]:
function product_except_self(nums):
    1. Initialize the result array of size n with 1s.
    2. Compute prefix products:   
         - prefix = 1   
         - for i in range(n):   
              result[i] = prefix   
              prefix *= nums[i]   
    3. Compute suffix products:   
         - suffix = 1   
         - for j in range(n-1, -1, -1):    
              result[j] *= suffix   
              suffix *= nums[j]   
    4. Return result   

**In the first loop**:
- The first (index 0) element will be 1 (first value of prefix)
- The second (index 1) element will be the product of 1 and the first element (1 * nums\[0\])
- The third (index 2) element will be the product of 1 * nums\[0\] * nums\[1\]
- ...
- The nearly last element (index n-2) will be the product of 1 * nums\[0\] * nums\[1\] * ... * nums\[n-3\]
- The last element (index n-1) will be the product of 1 * nums\[0\] * nums\[1\] * ... * nums\[n-2\]
- You can see that the last element has the value we need

**In the second loop**:
- Note that j starts with n-1, not 0 as the first loop, so result\[j\] in the first iteration is result\[n-1\]
- The last element has the value we need (mentioned above), so it multiplies by 1
- Gradually, the suffix will be the product of 1 * nums\[n-1\] * numns\[n-2\]..... 
- The nearly last element (index n-2) has the value: product of 1 * nums\[0\] * nums\[1\] * ... * nums\[n-3\], now it multiplies by result\[n-1\] * 1
- ...
- Until the end of the loop, it will be done.
- The second loop will loop from n-1 to 0. In the last iteration of this loop, the suffix now is the product of nums\[n-1\] *  nums \[n-2\] * ... * nums\[1\] (note that nums\[1\], not nums\[0\])

#### Code - Python

In [1]:
def product_except_self(nums):
    n = len(nums)
    result = [1] * n

    prefix = 1
    for i in range(n):
        result[i] = prefix
        prefix *= nums[i]

    suffix = 1
    for j in range(n-1, -1, -1):
        result[j] *= suffix
        suffix *= nums[j]
    return result

In [2]:
# Example Usage
print(product_except_self([1, 2, 3, 4, 5])) 
print(product_except_self([4, 7, 3]))       

[120, 60, 40, 30, 24]
[21, 12, 28]


#### Code - Golang

In [19]:
package main

import "fmt"

func productExceptSelf(nums []int) []int {
    n := len(nums)
    result := make([]int, n)
    
    prefix := 1
    for i:=0; i<n; i++ {
        result[i] = prefix
        prefix *= nums[i]
    }

    suffix := 1
    for j:=n-1; j>=0; j-- {
        result[j] *= suffix
        suffix *= nums[j]
    }
    
    return result
}

func main() {
    test1 := []int{1,2,3,4,5}
    test2 := []int{4, 7, 3}
    fmt.Println(productExceptSelf(test1))
    fmt.Println(productExceptSelf(test2))
}

[120 60 40 30 24]
[21 12 28]


## Problem 2: Locate smallest window to be sorted

Given an array of integers that are **out of order**, determine the bounds of the smallest window that must be sorted in order for the entire array to be sorted. For example, given \[3 , 7 , 5 , 6 , 9\], you should return (1, 3).

#### 🧩 The goal
We have an array (a list of numbers), and it’s **almost** sorted — but **some part in the middle** is out of order.   
We want to find the smallest section (window) that we would need to sort, so that after sorting **just that part**, the whole array becomes sorted.   
We can see in the above array, only \[7, 5, 6\] is not sorted, the indexes of this array are from 1 to 3 => return (1, 3)

#### ✅ Intuition:
1. Find the first element from the left that is out of order.
2. Find the first element from the right that is out of order.
3. The range between these two indices is the window we must sort. But to be precise:
    - We also need to expand the window if there are elements outside that are affected by the min/max of the subarray.

#### Step-by-Step Approach
Example: `[3, 7, 5, 6, 9]`   

1. Scan from left → right   
Find the first index where the array is not increasing:
    - 3 < 7 ✅
    - 7 > 5 ❌ → Left boundary candidate = 1
2. Scan from right → left   
Find the first index where the array is not decreasing:
    - 9 > 6 ✅
    - 6 < 5 ❌ → Right boundary candidate = 3
3. Find min and max of the subarray [7, 5, 6] → (min=5, max=7).
4. Expand boundaries:
    - Move left leftward if there are elements > min (none here).
    - Move right rightward if there are elements < max (none here).
    - ✅ Final window: (1, 3)

The example does not need step 4. If an array is \[1, 10, 5, 6, 15\] and we stop at step 3, we will have the unsorted array \[10, 5\].   
But if we sort this sub-array, we will have \[1, 5, 10, 6, 15\], and it's not correct.   
So if there are elements < max, we will have to move rightward. In this case, max(\[5, 10\]) is 10, it's bigger than 6 => move rightward.   
New sub-array is \[5, 10, 6\], max of it is 10 < 15 => no need to move rightward anymore.   
And min(\[5, 10\]) is 5 > 1, no need to move leftward.   
The final answer is (1, 3), not (1, 2).

#### Pseudo Code

In [ ]:
function find_window_to_sort(nums):
    left, right = None, None
    # Find left boundary
    for i in range(len(nums) - 1):
        if nums[i] > nums[i + 1]:
            left = i
            break

    if left is None:
        return (0, 0)  # Already sorted

    # Find right boundary
    for j in range(len(nums) - 1, 0, -1):
        if nums[j] < nums[j - 1]:
            right = j
            break

    min_val = min(nums[left:right+1])
    max_val = max(nums[left:right+1])

    # Expand left
    while left > 0 and nums[left - 1] > min_val:
        left -= 1

    # Expand right
    while right < len(nums) - 1 and nums[right + 1] < max_val:
        right += 1

    return (left, right)

#### Code - Python

In [3]:
def find_window_to_sort(nums):
    n = len(nums)
    left, right = None, None

    # Find first out-of-order from left
    for i in range(n - 1):
        if nums[i] > nums[i + 1]:
            left = i
            break

    if left is None:
        return (0, 0)  # Already sorted

    # Find first out-of-order from right
    for j in range(n - 1, 0, -1):
        if nums[j] < nums[j - 1]:
            right = j
            break

    # Find min and max in window
    window_min = min(nums[left:right + 1])
    window_max = max(nums[left:right + 1])

    # Expand left boundary
    while left > 0 and nums[left - 1] > window_min:
        left -= 1

    # Expand right boundary
    while right < n - 1 and nums[right + 1] < window_max:
        right += 1

    return (left, right)

# Example Usage
print(find_window_to_sort([3, 7, 5, 6, 9]))  # Output: (1, 3)
print(find_window_to_sort([1, 2, 3, 4, 5]))  # Output: (0, 0)
print(find_window_to_sort([1, 10, 8, 5, 6, 15]))  # Output: (1, 4)

(1, 3)
(0, 0)
(1, 4)


#### Code - Golang

In [1]:
package main

import (
	"fmt"
)

func findWindowToSort(nums []int) (int, int) {
	n := len(nums)
	left, right := -1, -1

	for i := 0; i < n-1; i++ {
		if nums[i] > nums[i+1] {
			left = i
			break
		}
	}
	if left == -1 {
		return 0, 0
	}

	for j := n - 1; j > 0; j-- {
		if nums[j] < nums[j-1] {
			right = j
			break
		}
	}

	windowMin, windowMax := nums[left], nums[left]
	for i := left; i <= right; i++ {
		if nums[i] < windowMin {
			windowMin = nums[i]
		}
		if nums[i] > windowMax {
			windowMax = nums[i]
		}
	}

	for left > 0 && nums[left-1] > windowMin {
		left--
	}
	for right < n-1 && nums[right+1] < windowMax {
		right++
	}

	return left, right
}

func main() {
	fmt.Println(findWindowToSort([]int{3, 7, 5, 6, 9})) // Output: 1, 3
}

1 3


### Problem 3: Calculate maximum subarray sum

## Problem 4: Find number of smaller elements to the right